In [1]:
import pandas as pd
import numpy as np
import pandasql as sqldf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sys,os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
def hot_encode_top (column, df, feat_count = 10):
    '''
    hot one encoding, limitiert auf die feat_count häufigsten features
    eines nominalen features um zu Hohe dimensionen zu vermeiden
    
    column: liste mit einem oder mehr Spaltennamen, die hot encoded werden sollen
    df: dataframe der die Datenbasis darstellt
    feat_count: Anzahl Spalten die für jede Spalte encoded werden
    '''
    df_ = df.copy(deep = True)
    
    for col in column:
        
        
        #nur die häufigsten feat_count Featues werden encoded
        encode_features = [x for x in df_[col].value_counts(ascending = False).head(feat_count).index]
        #if col == 'status_clean':
        #    encode_features = ['lapped', 'Finished', 'DNF']
        for feature in encode_features:
            col_feature = col + '_'+str(feature)
            #dort wo feature nicht dem encode feature entspricht wird eine 0 gesetzt
            df_[col_feature] = df_.where(df_[col] == feature, other = 0)[col]
            #encode feature selbst wird in dataframe durch eine 1 ersetzt
            df_[col_feature].replace(feature, 1, inplace = True)
        
        #löschen der nun "bereinigten" Spalte
        del df_[col]
        
    return df_

In [3]:
if os.path.exists('mit_dummies/sliced_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/mit_dummies/sliced_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    sliced_races = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('mit_dummies/sliced_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('mit_dummies/sliced_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        sliced_races[f] = df
    print('Einlesen der sliced Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
    
if os.path.exists('mit_dummies/split_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/mit_dummies/split_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    split_by_race = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('mit_dummies/split_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('mit_dummies/split_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        split_by_race[f] = df
    print('Einlesen der split Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')

Einlesen der sliced Dateien erfolgreich
Einlesen der split Dateien erfolgreich


In [5]:
t = sliced_races[841]
t.head()

,raceId,year,grandprix_name,lap_number,lap_position,lap_in_milliseconds,driver_fullname,podium_position,constructor_name,stop_binary,...,constructorId_206.0,constructorId_207.0,constructorId_208.0,constructorId_209.0,constructorId_210.0,status_clean_DNF,status_clean_Finished,status_clean_lapped,driverId,sum_milliseconds_pro_lap
0,841.0,2011.0,Australian Grand Prix,1.0,2.0,100573.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,100573.0
1,841.0,2011.0,Australian Grand Prix,2.0,2.0,93774.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,194347.0
2,841.0,2011.0,Australian Grand Prix,3.0,2.0,92900.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,287247.0
3,841.0,2011.0,Australian Grand Prix,4.0,2.0,92582.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,379829.0
4,841.0,2011.0,Australian Grand Prix,5.0,2.0,92471.0,Lewis Hamilton,2.0,McLaren,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,472300.0


In [ ]:
#shufflen der Rennen
import random 
keys = list(sliced_races.keys())
random.shuffle(keys)
sliced_shuffled = {}
for key in keys:
    sliced_shuffled[key] = sliced_races[key]


#columns werden definiert, die erst einmal nicht beachtet werden sollen
nogo_columns = ['podium_position', 'raceId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', #'total_laps',
               #'status_clean', 'constructorId',
                'total_milliseconds',
               'lap_in_milliseconds','year','driverId',
               'constructorId', 'status_clean','circuitId']

#definieren eines linearen Regressionsmodells
reg = LinearRegression()

train_break = 90
counter = 0
for raceId, race in sliced_shuffled.items():
    full = pd.DataFrame(columns = race.columns)
    if counter == train_break:
        break
    else:
        last_lap = max(race.lap_number.tolist())
        for did in race.driverId.unique():
            driver_df = race.where(np.logical_and(race.lap_number == last_lap, race.driverId == did)).dropna(how = 'all')
            full = full.append(driver_df)
            
            
        y = np.array(full['total_milliseconds'])
        cols = [col for col in full.columns if col not in nogo_columns]
        race = full[cols]
        
        #race = hot_encode_top(['driverId','status_clean', 'constructorId'],race):
        
        rows = [row for i, row in race.iterrows()]
        X = np.array(rows)
        
        #fitten der Daten auf das lineare Modell
        reg = reg.fit(X, y)
        A = reg.predict(X)
        mse = mean_squared_error(A, y)
        print('mse:', mse)
        counter+=1

In [20]:
max(sliced_shuffled[841]['race_completion'])

0.6896551724137931

In [61]:
#shufflen der Rennen
import random 
print('---shuffling---')
keys = list(sliced_races.keys())
random.shuffle(keys)
sliced_shuffled = {}
for key in keys:
    sliced_shuffled[key] = sliced_races[key]
    
#columns werden definiert, die erst einmal nicht beachtet werden sollen
nogo_columns = ['podium_position', 'raceId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', #'total_laps',
               #'status_clean', 'constructorId',
                'total_milliseconds',
               'lap_in_milliseconds','year','driverId',
               'constructorId', 'status_clean','circuitId']
    
#definieren eines linearen Regressionsmodells
reg = LinearRegression()
print('---create Linear Regression Model---')


train_break = 90
counter = 0
print('---start Model training---')
#for key, value in sliced_shuffled.items():
#    print(str((counter+1)/(train_break+1)*100)+'% trained')
#    if counter == train_break:
#        break
#    else:
#        for did in value.driverId.unique():
#            temp = value.where(value.driverId == did).dropna(how = "all")
#            y = np.array(temp["total_milliseconds"])
#            print('y_training:',y[0])
#            #temp_y = temp_y[0]
#            cols = [col for col in temp.columns if col not in nogo_columns]
#            temp_x = temp[cols]
#            
#            rows = [row for i, row in temp_x.iterrows()]
#            X = np.array(rows)
test_cols = sliced_shuffled[841].columns
cols = [col for col in test_cols if col not in nogo_columns]
X = pd.DataFrame(columns = cols)
y = pd.Series()

for key,value in sliced_shuffled.items():
    y_ = value['total_milliseconds']
    #print(y_)
    y = y.append(y_)
    print(y)
    value = value[cols]
    X = X.append(value)

X.reset_index(inplace = True, drop = True)
#y.reset_index(drop = True, inplace = True)
#X['y'] = y['total_milliseconds']
#X.dropna(how = 'all',inplace = True)

#y = np.array(X['y'])#.reshape(-1, 1)

y = np.array(y).reshape(-1,1)
#del X['y']
X_cols = [col for col in X.columns if col != 'y']
X = X[X_cols]
#rows = [row for i,row in X.iterrows()]

X_ = np.array(X)#.reshape(1, -1)
#print(X_)
print(y)

print('vor fit')
reg = reg.fit(X_, y)
#beta_hat = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
#print(beta_hat)
#reg = reg.fit(X, y)
#print('Nach fit und vor prediction')
A = reg.predict(X_)
#print('vor mse')
mse = mean_squared_error(A, y)
print('mse:', mse)
counter+=1
        

---shuffling---
---create Linear Regression Model---
---start Model training---
0      7495124.0
1      7495124.0
2      7495124.0
3      7495124.0
4      7495124.0
         ...    
597    7477326.0
598    7477326.0
599    7477326.0
600    7477326.0
601    7477326.0
Length: 602, dtype: float64
0       7.495124e+06
1       7.495124e+06
2       7.495124e+06
3       7.495124e+06
4       7.495124e+06
            ...     
996     6.442273e+06
997     6.442273e+06
998     6.442273e+06
999     6.442273e+06
1000    6.442273e+06
Length: 1603, dtype: float64
0      7.495124e+06
1      7.495124e+06
2      7.495124e+06
3      7.495124e+06
4      7.495124e+06
           ...     
948    6.589044e+06
949    6.589044e+06
950    6.589044e+06
951    6.589044e+06
952    6.589044e+06
Length: 2556, dtype: float64
0      7495124.00
1      7495124.00
2      7495124.00
3      7495124.00
4      7495124.00
          ...    
622    5001018.18
623    5001018.18
624    5001018.18
625    5001018.18
626    5001018.1

Length: 31748, dtype: float64
0      7.495124e+06
1      7.495124e+06
2      7.495124e+06
3      7.495124e+06
4      7.495124e+06
           ...     
721    6.045822e+06
722    6.045822e+06
723    6.045822e+06
724    6.045822e+06
725    6.045822e+06
Length: 32474, dtype: float64
0       7.495124e+06
1       7.495124e+06
2       7.495124e+06
3       7.495124e+06
4       7.495124e+06
            ...     
997     5.548001e+06
998     5.548001e+06
999     5.548001e+06
1000    5.548001e+06
1001    5.548001e+06
Length: 33476, dtype: float64
0      7.495124e+06
1      7.495124e+06
2      7.495124e+06
3      7.495124e+06
4      7.495124e+06
           ...     
849    4.906006e+06
850    4.906006e+06
851    4.906006e+06
852    4.906006e+06
853    4.906006e+06
Length: 34330, dtype: float64
0      7495124.0
1      7495124.0
2      7495124.0
3      7495124.0
4      7495124.0
         ...    
785    3937712.0
786    3937712.0
787    3937712.0
788    3937712.0
789    3937712.0
Length: 35120, dtype: 

Length: 60971, dtype: float64
0       7.495124e+06
1       7.495124e+06
2       7.495124e+06
3       7.495124e+06
4       7.495124e+06
            ...     
1024    6.189104e+06
1025    6.189104e+06
1026    6.189104e+06
1027    6.189104e+06
1028    6.189104e+06
Length: 62000, dtype: float64
0       7.495124e+06
1       7.495124e+06
2       7.495124e+06
3       7.495124e+06
4       7.495124e+06
            ...     
1051    6.162766e+06
1052    6.162766e+06
1053    6.162766e+06
1054    6.162766e+06
1055    6.162766e+06
Length: 63056, dtype: float64
0      7.495124e+06
1      7.495124e+06
2      7.495124e+06
3      7.495124e+06
4      7.495124e+06
           ...     
947    6.690916e+06
948    6.690916e+06
949    6.690916e+06
950    6.690916e+06
951    6.690916e+06
Length: 64008, dtype: float64
0      7.495124e+06
1      7.495124e+06
2      7.495124e+06
3      7.495124e+06
4      7.495124e+06
           ...     
665    5.800277e+06
666    5.800277e+06
667    5.800277e+06
668    5.800277e+0

Length: 88650, dtype: float64
0      7.495124e+06
1      7.495124e+06
2      7.495124e+06
3      7.495124e+06
4      7.495124e+06
           ...     
804    6.167772e+06
805    6.167772e+06
806    6.167772e+06
807    6.167772e+06
808    6.167772e+06
Length: 89459, dtype: float64
0      7495124.0
1      7495124.0
2      7495124.0
3      7495124.0
4      7495124.0
         ...    
786    5615553.0
787    5615553.0
788    5615553.0
789    5615553.0
790    5615553.0
Length: 90250, dtype: float64
0      7.495124e+06
1      7.495124e+06
2      7.495124e+06
3      7.495124e+06
4      7.495124e+06
           ...     
749    4.872230e+06
750    4.872230e+06
751    4.872230e+06
752    4.872230e+06
753    4.872230e+06
Length: 91004, dtype: float64
0      7495124.0
1      7495124.0
2      7495124.0
3      7495124.0
4      7495124.0
         ...    
719    2885520.0
720    2885520.0
721    2885520.0
722    2885520.0
723    2885520.0
Length: 91728, dtype: float64
0      7495124.0
1      7495124.0
2 

In [ ]:
print('---start Model testing---')  
counter = 0
count_richtig = 0
p_counter = 1

for raceId, value in sliced_shuffled.items():
    
    if counter == train_break:
        print(str(p_counter/(len(sliced_shuffled.keys())-train_break)*100)+'% tested')
        p_counter += 1
        #iterieren über alle fahrer des rennens
        for did in value.driverId.unique():
            temp = value.where(value.driverId == did).dropna(how = "all")
            y = np.array(temp["total_milliseconds"])
            print('y:',y[0])
            #temp_y = temp_y[0]
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            
            rows = [row for i, row in temp_x.iterrows()]
            X = np.array(rows)
            A = reg.predict(X)
            print('Prediction:',A)
            print('Richtige Zeit:',y)
            mse = mean_squared_error(A, y)
            print('mse:', mse)

        
        
       # ##gewinner = full.where(full.podium_position == 1).dropna(how='all')['driver_fullname'].tolist()[0]
       # #print('Gewinner:',gewinner)
       # y = np.array(full['total_milliseconds'])
       # cols = [col for col in full.columns if col not in nogo_columns]
       # race_ = full[cols]
       # #race_ = hot_encode_top(['driverId','status_clean', 'constructorId'],race_)
       # 
       # rows = [row for i, row in race_.iterrows()]
       # X = np.array(rows)
       # 
       # A = reg.predict(X)
       # 
       # full['prediction'] = A
       # #print('A', A)
       # #top_10_pred = get_top_10(full)
       # pred_gewinner = full.where(full.prediction == min(full.prediction)).dropna(how='all')['driver_fullname'].tolist()[0]
       # print('predicteter Gewinner:\n', pred_gewinner)
       # mse = mean_squared_error(A, y)
       # print('mse:', mse, '\n\n')
       # 
       # if pred_gewinner == gewinner:
       #     print('TREFFER')
       #     count_richtig += 1
    else:
        counter += 1
        
#print('Anzahl richtiger 1ter Plätze:', count_richtig)

In [50]:
for column in X.columns:
    for i in range(len(X)):
        if np.isnan(X.loc[i,column])== True:
            print(column, i)

In [13]:
print('---start Model testing---')  
counter = 0
count_richtig = 0
p_counter = 1

for raceId, value in sliced_shuffled.items():
    
    if counter == train_break:
        print(str(p_counter/(len(sliced_shuffled.keys())-train_break)*100)+'% tested')
        p_counter += 1
        #iterieren über alle fahrer des rennens
        for did in value.driverId.unique():
            temp = value.where(value.driverId == did).dropna(how = "all")
            y = np.array(temp["total_milliseconds"])
            #temp_y = temp_y[0]
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            
            rows = [row for i, row in temp_x.iterrows()]
            X = np.array(rows)
            A = reg.predict(X)
            print('Prediction:',A)
            print('Richtige Zeit:',y)
            mse = mean_squared_error(A, y)
            print('mse:', mse)

        
        
       # ##gewinner = full.where(full.podium_position == 1).dropna(how='all')['driver_fullname'].tolist()[0]
       # #print('Gewinner:',gewinner)
       # y = np.array(full['total_milliseconds'])
       # cols = [col for col in full.columns if col not in nogo_columns]
       # race_ = full[cols]
       # #race_ = hot_encode_top(['driverId','status_clean', 'constructorId'],race_)
       # 
       # rows = [row for i, row in race_.iterrows()]
       # X = np.array(rows)
       # 
       # A = reg.predict(X)
       # 
       # full['prediction'] = A
       # #print('A', A)
       # #top_10_pred = get_top_10(full)
       # pred_gewinner = full.where(full.prediction == min(full.prediction)).dropna(how='all')['driver_fullname'].tolist()[0]
       # print('predicteter Gewinner:\n', pred_gewinner)
       # mse = mean_squared_error(A, y)
       # print('mse:', mse, '\n\n')
       # 
       # if pred_gewinner == gewinner:
       #     print('TREFFER')
       #     count_richtig += 1
    else:
        counter += 1
        
print('Anzahl richtiger 1ter Plätze:', count_richtig)

---start Model testing---
2.127659574468085% tested
Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889]
Richtige Zeit: [5062990. 5062990. 5062990. 5062990. 5062990. 5062990. 5062990. 5062990.
 5062990. 5062990. 5062990. 5062990. 5062990. 5062990. 5062990. 5062990.
 5062990. 5062990. 5062990. 5062990. 5062990. 5062990. 5062990. 5062990.
 5062990. 5062990. 5062990. 5062990. 5062990. 5062990.]
mse: 1326520896065.2368
Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214

Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889]
Richtige Zeit: [5049930. 5049930. 5049930. 5049930. 5049930. 5049930. 5049930. 5049930.
 5049930. 5049930. 5049930. 5049930. 5049930. 5049930. 5049930. 5049930.
 5049930. 5049930. 5049930. 5049930. 5049930. 5049930. 5049930. 5049930.
 5049930. 5049930. 5049930. 5049930. 5049930. 5049930.]
mse: 1356775088403.015
Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 62147

Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889]
Richtige Zeit: [5929688.76923077 5929688.76923077 5929688.76923077 5929688.76923077
 5929688.76923077 5929688.76923077 5929688.76923077 5929688.76923077
 5929688.76923077 5929688.76923077 59296

Prediction: [6214736.88888889]
Richtige Zeit: [160999.]
mse: 36647742427368.914
Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889]
Richtige Zeit: [5945569.78125 5945569.78125 5945569.78125 5945569.78125 5945569.78125
 5945569.78125 5945569.7812

Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889]
Richtige Zeit: [7130416.06666667 7130416.06666667 7130416.06666667 7130416.06666667
 7130416.06666667 7130416.06666667 7130416.06666667 7130416.06666667
 7130416.06666667 7130416.06666667 7130416.06666667 7130416.06666667
 7130416.06666667 7130416.06666667 71304

...

Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889]
Richtige Zeit: [5274976. 5274976. 5274976. 5274976. 5274976. 5274976. 5274976. 5274976.
 5274976. 5274976. 5274976. 5274976. 5274976. 5274976

Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889]
Richtige Zeit: [5467777.91304348 5467777.91304348 5467777.91304348 5467777.91304348
 5467777.91304348 5467777.91304348 5467777.91304348 54677

Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889]
Richtige Zeit: [5116124. 5116124. 5116124. 5116124. 5116124. 5116124. 5116124. 5116124.
 5116124. 5116124. 5116124. 5116124. 5116124. 5116124. 5116124. 5116124.
 5116124. 5116124. 5116124. 5116124. 5116124. 5116124. 5116124. 5116124.
 5116124. 5116124. 5116124. 5116124. 5116124. 5116124. 5116124. 5116124.
 5116124. 5116124. 5116124. 5116124.]
mse: 1206950279632.79

Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889]
Richtige Zeit: [5882398. 5882398. 5882398. 5882398. 5882398. 5882398. 5882398. 5882398.
 5882398. 5882398. 5882398. 5882398. 5882398. 5882398. 5882398. 5882398.
 5882398. 5882398. 5882398. 5882398. 5882398. 5882398. 5882398. 5882398.
 5882398. 5882398. 5882398. 5882398. 5882398. 5882398. 5882398. 5882398.
 588239

Prediction: [6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889 6214736.88888889
 6214736.88888889 6214736.88888889 6214736.88888889]
Richtige Zeit: [5906342.4 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4
 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4
 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4
 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4 5906342.4
 5906342.4 5906

In [ ]:
def get_top_10(df):
    df_ = df.copy(deep = True)
    #print('in get top 10', df_.head())
    position = pd.DataFrame(columns = ['position', 'name'])
    pos = 1
    for d in range(len(df_)):
        idx = df_.where(df_.prediction == min(df_.prediction)).dropna(how = 'all').index
        winner = df_.loc[idx,'driver_fullname']
        position.loc[d,:] = [str(pos), winner]
        pos += 1
        df_ = df_.drop(index = idx)
        
    return position